In [ ]:
%%capture --no-stderr
%pip install -U langgraph langchain-community langchain-anthropic tavily-python pandas  langchain_groq langchain_fireworks

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Complex_Tool_Calling"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
os.environ["SCRAPYBARA_API_KEY"] = userdata.get("SCRAPYBARA_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["FIREWORKS_API_KEY"] = userdata.get("FIREWORKS_API_KEY")
from langchain import hub
from langchain_groq import ChatGroq
from langchain.agents import create_openai_functions_agent

llm = ChatGroq(model_name = "llama3-70b-8192",api_key = os.environ["GROQ_API_KEY"])



In [ ]:
! pip install trustcall

In [ ]:
import uuid

from typing import Optional,List

from pydantic import BaseModel,Field

In [ ]:
class Person(BaseModel):
  """Someone That user want to ask"""

  name:str
  relationship:str = Field(description="How thwy Related To User")

  notes:List[str] = Field(
      description="Memories and Observation of Person"
  )


initial_people = [
     Person(
        name = "Emma Thompson",
        relationship = "College Friend",
        notes=["Loves hiking", "Works in marketing", "Has a dog named Max"],
    ),

     Person(
        name = "Michael Chen",
        relationship = "Coworker",
        notes=["Great at problem-solving", "Vegetarian", "Plays guitar"],
    ),


    Person(
        name="Sarah Johnson",
        relationship="Neighbor",
        notes=["Has two kids", "Loves gardening", "Makes amazing cookies"],
    ),

]


existing_data = [
    (str(i), "Person", person.model_dump()) for i, person in enumerate(initial_people)
]




In [ ]:
conversation = """
Me: I ran into Emma Thompson at the park yesterday. She was walking her new puppy, a golden retriever named Sunny. She mentioned she got promoted to Senior Marketing Manager last month.
Friend: That's great news for Emma! How's she enjoying the new role?
Me: She seems to be thriving. Oh, and did you know she's taken up rock climbing? She invited me to join her at the climbing gym sometime.
Friend: Wow, rock climbing? That's quite a change from hiking. Speaking of friends, have you heard from Michael Chen recently?
Me: Actually, yes. We had a video call last week. He's switched jobs and is now working as a Data Scientist at a startup. He's also mentioned he's thinking of going vegan.
Friend: That's a big change for Michael! Both career and diet-wise. How about your neighbor, Sarah? Is she still teaching?
Me: Sarah's doing well. Her kids are growing up fast - her oldest just started middle school. She's still teaching, but now she's focusing on special education. She's really passionate about it.
Friend: That's wonderful. Oh, before I forget, I wanted to introduce you to my cousin who just moved to town. Her name is Olivia Davis, she's a 27-year-old graphic designer. She's looking to meet new people and expand her social circle. I thought you two might get along well.
Me: That sounds great! I'd love to meet her. Maybe we could all get together for coffee next week?
Friend: Perfect! I'll set it up. Olivia loves art and is always sketching in her free time. She also volunteers at the local animal shelter on weekends.
"""



In [ ]:
from trustcall import create_extractor

In [ ]:
extractor = create_extractor(
    llm,
    tools=[Person],
    tool_choice="any",
    enable_inserts=True,
)



In [ ]:
extractor.invoke(
    {
        "messages":[
            {
                "role":"user",
                "content": f"Update existing person records and create new ones based on the following conversation:\n\n{conversation}",

            }
        ],
        "existing":existing_data,
    }
)

ERROR:extraction:Could not apply patch: 'path' with '-' can't be applied to 'replace' operation
ERROR:extraction:Could not apply patch: 'path' with '-' can't be applied to 'replace' operation


{'messages': [AIMessage(content='', additional_kwargs={'updated_docs': {'call_1t13': '2'}}, response_metadata={}, id='3f9ddfe8-6d60-4407-b3a0-ca8c4ae8f781', tool_calls=[{'name': 'Person', 'args': {'name': 'Sarah Johnson', 'relationship': 'Neighbor', 'notes': ['Has two kids', 'Loves gardening', 'Makes amazing cookies', 'Teaches special education']}, 'id': 'call_1t13', 'type': 'tool_call'}, {'name': 'Person', 'args': {'name': 'Olivia Davis', 'relationship': 'Friend of a Friend', 'notes': ['Loves art', 'Enjoys sketching', 'Volunteers at the local animal shelter']}, 'id': 'call_e60c', 'type': 'tool_call'}])],
 'responses': [Person(name='Sarah Johnson', relationship='Neighbor', notes=['Has two kids', 'Loves gardening', 'Makes amazing cookies', 'Teaches special education']),
  Person(name='Olivia Davis', relationship='Friend of a Friend', notes=['Loves art', 'Enjoys sketching', 'Volunteers at the local animal shelter'])],
 'response_metadata': [{'id': 'call_1t13', 'json_doc_id': '2'},
  {'id